Timing test suite.

In the following cell: setting up
* imports
* reading data
* defining different functions that we want to time


In [1]:
import numpy as np
invm = np.linalg.inv
import xarray as xr
from blockinv_iterative import block_iter_inv
from blockinv_iterative_symmetric import block_inv_s


ds_1 = xr.open_dataset("data/regions_verify_isotope_202112_cov.nc")
bio_1 = ds_1["covariance_bio"]
anth_1 = ds_1["covariance_anth"]

ds_2 = xr.open_dataset("data/regions_verify_202104_cov.nc")
bio_2 = ds_2["covariance_bio"]
anth_2 = ds_2["covariance_anth"]

M = bio_2.to_numpy()



Matrix inv accuracy test

In [2]:
k = int(len(M)/2)

inv1 = invm(M)
inv2 = block_iter_inv(M, 5)
#inv4 = block_inv_s(M, k)

print(np.allclose(inv1, inv2))

True


Timer

In [3]:


#time_d = %timeit -r 10 -n 10 -o invm(M)
#time_f = %timeit -r 10 -n 10 -o block_iter_inv_f(M,5)
#time_w = %timeit -r 10 -n 10 -o block_iter_inv_w(M,5)

time_fd = %timeit -r 10 -n 10 -o block_iter_inv(M,5)




7.52 s ± 1.31 s per loop (mean ± std. dev. of 10 runs, 10 loops each)


Timer results 

As far as I've understood, the timer runs -r <R> number of times and loops within each run -n <N> times, picking the minimum of those N loops.
So, to get accurate results, N should be rather large! 

with M = bio_2.to_numpy(): 

default invm: 8.54 s ± 1.48 s per loop (mean ± std. dev. of 10 runs, 10 loops each)

block_iter_inv(M,5):
    * with for-loop and block_inv: 6.79 s ± 432 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
    * with for-loop and default inv: 7.52 s ± 1.31 s per loop (mean ± std. dev. of 10 runs, 10 loops each)


block_iter_inv_w(M,5), version using while-loop and block_inv: 7.1 s ± 392 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)